In [1]:
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from xppbe.Model.Solutions_utils import Solution_utils
from xppbe.Model.PDE_Model import PBE
from xppbe import xppbe_path

file_path = os.getcwd()

In [2]:

molecule = '1ubq'


Rs = {'born_ion': 1,
      'methanol': 2.54233,
      'methanol_E': 2.54233,
      'arg': 5.9695745,
      '1ubq': 24.697,
      'sphere': 1.2,
      'sphere_+1-1': 1.2,
      'sphere_+1+1': 1.2,
      'sphere_+2-1': 1.2,
      'sphere_+1_D': 1.2,
      'sphere_Nqs': 1.2}

Model = Solution_utils()

Model.main_path = xppbe_path
Model.molecule_path = os.path.join(xppbe_path,'Molecules',molecule)
Model.DTYPE = 'float32'

Model.domain_properties = {
        'molecule': molecule,
        'epsilon_1':  2,
        'epsilon_2': 80,
        'kappa': 0.125,
        'T' : 300 
        }

for key in Model.domain_properties:
      setattr(Model, key, Model.domain_properties[key])


PBE.get_charges(Model)


In [3]:
Model.q_list

[Charges(q=0.1592, x_q=array([ -3.1999,  -4.9891, -15.3285]), r_q=1.824, atom_name='N', res_name='MET', res_num='1', ion_r_explode=3.5),
 Charges(q=0.0221, x_q=array([ -4.2739,  -4.0061, -15.1005]), r_q=1.908, atom_name='CA', res_name='MET', res_num='1', ion_r_explode=3.5),
 Charges(q=0.6123, x_q=array([ -3.6269,  -2.7801, -14.4115]), r_q=1.908, atom_name='C', res_name='MET', res_num='1', ion_r_explode=3.5),
 Charges(q=-0.5713, x_q=array([ -2.6539,  -2.9561, -13.6795]), r_q=1.6612, atom_name='O', res_name='MET', res_num='1', ion_r_explode=3.5),
 Charges(q=0.0865, x_q=array([ -5.4279,  -4.5391, -14.2935]), r_q=1.908, atom_name='CB', res_name='MET', res_num='1', ion_r_explode=3.5),
 Charges(q=0.0334, x_q=array([ -5.1869,  -4.5591, -12.8085]), r_q=1.908, atom_name='CG', res_name='MET', res_num='1', ion_r_explode=3.5),
 Charges(q=-0.2774, x_q=array([ -6.6099,  -5.4601, -12.0385]), r_q=2.0, atom_name='SD', res_name='MET', res_num='1', ion_r_explode=3.5),
 Charges(q=-0.0341, x_q=array([ -6.0

In [4]:
dR = 0.4
R_mol = Rs[molecule]
Rmin = dR + R_mol
Rmax = Rmin + 3

Model.DTYPE = 'float32'

### Calculate Solvation energy

In [5]:
phi_q = Model.Spherical_Harmonics(Model.x_qs,'molecule',R_mol, N=30)
print(phi_q)
PBE.solvation_energy_phi_qs(Model,phi_q)

[-0.0262375  -0.07895157  0.00366047 -0.05481606 -0.00050707]


-548.7745934585375

### Create data_known file

In [7]:

def calculate_print_phi(domain,method, num_points, file):

    R = R_mol if domain == 'molecule' else Rmax if domain == 'solvent' else None
    x = np.random.uniform(-R, R, num_points)
    y = np.random.uniform(-R, R, num_points)
    z = np.random.uniform(-R, R, num_points)
    r = np.sqrt(x**2+y**2+z**2)
    X = np.stack([x,y,z], axis=1).astype(np.float32)

    phi_values = Model.phi_known(method,'phi',X,domain,R_mol)

    if domain == 'molecule':
        for i in range(num_points):
            condition = 1 if r[i] <= R_mol else 2
            if r[i]<R_mol:
                file.write(f"{condition} {x[i]} {y[i]} {z[i]} {phi_values[i]}\n")

    elif domain == 'solvent':
        for i in range(num_points):
            condition = 1 if r[i] <= R_mol else 2
            if r[i]>Rmin and r[i]<Rmax:
                file.write(f"{condition} {x[i]} {y[i]} {z[i]} {phi_values[i]}\n")

In [11]:
with open(os.path.join(file_path,molecule,'data_known.dat'), 'w') as file:
    calculate_print_phi('solvent','G_Yukawa',2500, file)
    # calculate_print_phi('molecule','Harmonic_spheres',300, file)

print('File created')

File created
